# Creating Scorecards and Trends and Putting them into an Organizer Topic Document
## Introduction
Algorithmically creating topics using Seeq's REST API can be done using any programming language.  This tutorial will walk through creating these object using Python 2.7 and Seeq's SDK.

The tutorial will show how to create a new Organizer containing a new Topic and modifying the text shown in the topic.  Once this has been accomplished, I recommend that you check out the _Embedding Metrics and Trends using the API_ tutorial to see how to create and include Seeq content.

Note that doing these kinds of operations through the API circumvents all of the front-end's safety checks.  It is **highly recommended** that you develop on a local machine and test all code thoroughly **before using your code on an active server**. You should probably also **back that server up**. As an example of what can go wrong, forgetting the `"`'s in the `workbook_input.data` JSON will render the server unreachable by all...and potentially unfixable!


## Creating and Embedding Scorecards and Trends
Creating both scorecards and trends is very similar.  It boils down to creating the desired content using the appropriate endpoint (only required when making the metric), setting Worksteps to show the content you want, then creating screenshots of those Worksteps.  The final piece of the puzzle is to embed those screenshots in the topic documents, which is done using many of the same tools that are shown in the _Creating Organizer Topics using the API_ tutorial.

The basic process for creating a scorecard or trend view and putting it into a document is:
1. Create a Workbench Analysis Worksheet to display the view.
1. Create a Metric for our scorecard based on a signal, condition, and aggregation method (or one of the other kinds of metrics).
1. Update the Worksheet to show the scorecard or trend view.
1. Generate a screenshot of the view based on the Date Range.
1. Embed the screenshot into the Organizer Topic document.

### Setting up the Environment

In [ ]:
# import the Seeq SDK for communicating with Seeq (I installed this from the egg distributed with Seeq and easy_install)
import seeq_sdk as sdk
# import Jinja2 for creating html templates to post in the document
import jinja2 as ji
# import uuid so that we can create UUID's (aka GUIDs) for our objects
from uuid import uuid4
# import json so that to parse objects
import json
# import datetime for working with times
from datetime import datetime as dt
# import copy so that we can use templates
import copy

### Connecting to Seeq
This is the basic stuff.  Note that these are dummy credentials for a local installation which will not work on your machine or in the cloud.

In [ ]:
# create the API client
api_client = sdk.ApiClient("http://localhost:34216/api", None, None)

# get the clent authoization and login
auth_input = sdk.AuthInputV1(username='seth.gilchrist@seeq.com', password='testpass')
auth_api = sdk.AuthApi(api_client)
auth_output = auth_api.login(body=auth_input)

### Creating a Workbench Analysis
The Workbench Analysis is created in a very similar manner to the Organizer Topic.  I will not go into detail on creating the analysis, as there is another how-to focused on that.  We will leave the Workstep uninitialized and create it after we have our Metric.

In [ ]:
# create the Analysis
analysis_input = sdk.WorkbookInputV1()
analysis_input.name = "API Created Analysis"
analysis_output = workbook_api.create_workbook(body=analysis_input)

# create the worksheet
worksheet_input = sdk.WorksheetInputV1()
worksheet_input.name = "API Created Worksheet"
worksheet_output = workbook_api.create_worksheet(workbook_id=analysis_output.id, body=worksheet_input)

# set the initial workbook state
analysis_property_input = sdk.PropertyInputV1()
analysis_property_input.unit_of_measure = 'string'
analysis_property_input.value = '{"version":1,"state":{"stores":{}}}'
analysis_state_output = items_api.set_property(id=analysis_output.id,
                                               property_name='workbookState',
                                               body=analysis_property_input)
print("Workbook ID: {0}\nWorksheet ID: {1}".format(analysis_output.id, worksheet_output.id))

### Creating the Scorecard Metric
To create the metric, we need to have a signal ID and a condition ID against which to perform our calculations, and a aggregation function that we want to use to summarize the signal over the condition's capsules.  I will get my IDs from the Seeq interface, but they could come from API generated signals and conditions as well.  To replicate my example, you can use the IDs of the signals and conditions defined below.

Signal ID: ID of the example data `Example >> Cooling Tower 1 >> Area A >> Compressor Power`

Condition ID: ID of a value search on the signal above with a formula of

`$a.validValues().valueSearch(1day, isGreaterThan(25), 0min, isLessThanOrEqualTo(25), 0min)`

Once we have these IDs, we can use the Metrics API endpoint to create our metric.  I will be aggregating over the maximum value of the signal during the capsules (hence `aggregation_function = maxValue()`), but other aggregators are available.  To find the formula for the one you are interested in, create a scorecard with your target aggregator and examine the items properties in Seeq.

Note that `scoped_to` provides search scope for our metric, it's best to scope it to our Workbench Analysis so that it doesn't pollute other search locations. Also, instantiating `threasholds` to an empty list is required.

In [ ]:
metric_input = sdk.ThresholdMetricInputV1()
metric_input.name = 'API Created Metric 1'
metric_input.measured_item = "5D9A8558-F592-4C3B-8361-D88FDD52F916"
metric_input.bounding_condition = "809E03A9-3949-4087-86D5-4B47FE3B2F8E"
metric_input.aggregation_function = "maxValue()"
metric_input.scoped_to = analysis_output.id
metric_input.thresholds = []

metric_api = sdk.MetricsApi(api_client)
metric_output = metric_api.create_threshold_metric(body=metric_input)
print(metric_output)

Now that the metric has been created, we will make a scorecard from it, which is done using a Workstep.  We will create a Workstep that shows the metric in a scorecard by applying appropriate values to two dateStores: the sqWorksheetStore, in which we set the `viewKey` to `SCORECARD` (an alternative being `TREND`) and an sqTrendMetricStore in which we tell Seeq how our Metric should be displayed.  You may want to play around with the `scorecardHeaders` to get them how you want.

In [ ]:
worksheet_store = '\
{\
    "tabsets": {\
        "sidebar": 1\
    },\
    "viewKey": "SCORECARD",\
    "browsePanelCollapsed": false,\
    "displayWidth": 850,\
    "displayHeight": 575,\
    "resizeEnabled": true\
}'

trend_metric_template = '\
{\
    "scorecardHeaders": {\
        "type": "start",\
        "format": "lll"\
    },\
    "scorecardColumns": [\
        {\
            "type": "name",\
            "backgroundColor": "#ffffff"\
        }],\
    "dataStatus": "itemDataPresent",\
    "warningCount": 0,\
    "warningLogs": [],\
    "items": [\
        {\
            "axisAlign": "B",\
            "axisAutoScale": true,\
            "lane": 2,\
            "rightAxis": false,\
            "dashStyle": "Solid",\
            "lineWidth": 1,\
            "autoDisabled": false,\
            "axisVisibility": true,\
            "yAxisConfig": {\
              "min": -1.14,\
              "max": 1.14\
            },\
            "yAxisMin": -1,\
            "yAxisMax": 1,\
            "sampleDisplayOption": "line",\
            "id": "ID",\
            "name": "name",\
            "selected": false,\
            "color": "#9D248F",\
            "scorecardOrder": 0\
        }]\
}'

trend_metric_json = json.loads(trend_metric_template)
trend_metric_json['items'][0]['id'] = metric_output.id
trend_metric_json['items'][0]['name'] = metric_output.name

data_store = '{{"sqWorksheetStore": {0}, "sqTrendMetricStore": {1} }}'.format(worksheet_store, json.dumps(trend_metric_json))

analysis_workstep_input = sdk.WorkstepInputV1()
analysis_workstep_input.data = '{{"version":23,"state":{{"stores": {0} }} }}'.format(data_store)

analysis_workstep_output = workbook_api.create_workstep(workbook_id=analysis_output.id,
                                                        worksheet_id=worksheet_output.id,
                                                        body=analysis_workstep_input)
print(analysis_workstep_output)

You can now surf over to Seeq, open the Analysis and see the scorecard that you've created.

### Creating a Trend View
Creating scorecads is all well and good, but what about creating a trend view?  If we know what signals and conditions comprise a scorecard, then we can setup a trend showing them as well.

This can be done 100% algorithmically, but I would recommend setting up the trend just how you like it, grabbing a copy of the worksheet dataStore, and using that as a template.  From there, you can swap signals in and out, but most likely you'll be modifying only the date range of the trend view.

A typical workflow for embedding both a scorecard and trend in a topic would be the following:
* create the metric $\checkmark$
* create a Worksheet showing the metric $\checkmark$
* generate a screenshot of it (our next step)
* update the Worksheet's Workstep to show the trend
* generate a screenshot of that
Using this workflow you have one Worksheet that is used for both the scorecard and trend views.

While it's slightly out of sequence, I'm going to create the trend-view workstep now so that we can do the screenshots in quick succession.

Including signals in a trend view is done using the `sqTrendSeriesStore`, and including the condition is done using the `sqTrendCapsuleSetStore`.  Setting the view parameters happens in a few places in the dataStore, but the most important ones are in the `sqWorksheetStore`.  Let's define our parameters for these stores.

In [ ]:
# We already have an sqWorksheetStore template, so we'll just update it.
worksheet_store_json = json.loads(worksheet_store)
worksheet_store_json['viewKey'] = "TREND"

# now for the other two
trend_series_store = '\
{\
    "items": [\
        {\
            "axisAlign": "A",\
            "axisAutoScale": true,\
            "lane": 1,\
            "rightAxis": false,\
            "dashStyle": "Solid",\
            "lineWidth": 1,\
            "autoDisabled": false,\
            "axisVisibility": true,\
            "yAxisConfig": {\
              "min": -1.2482005542500003,\
              "max": 37.00042770425\
            },\
            "yAxisMin": 0.0029228,\
            "yAxisMax": 35.74930435,\
            "sampleDisplayOption": "line",\
            "id": "",\
            "name": "",\
            "selected": false,\
            "color": "#E1498E",\
            "valueUnitOfMeasure": ""\
          }\
    ],\
        "editingId": null,\
        "previewSeriesDefinition": {}\
}'
trend_series_json = json.loads(trend_series_store)
# everything we need to know about the series we want to show is in metric_output, except for the unit of measure.
# However, the metric has the same UOM as the measured item, so we'll just use that.
trend_series_json['items'][0]['id'] = metric_output.measured_item.id
trend_series_json['items'][0]['name'] = metric_output.measured_item.name
trend_series_json['items'][0]['valueUnitOfMeasure'] = metric_output.value_unit_of_measure

trend_capsule_set_store = '\
{\
    "items": [\
        {\
            "autoDisabled": false,\
            "id": "ID",\
            "name": "",\
            "selected": false,\
            "color": "#00A2DD"\
         }\
    ]\
}'
trend_capsule_set_json = json.loads(trend_capsule_set_store)
# again, let's get all the info we need from metric_output
trend_capsule_set_json['items'][0]['id'] = metric_output.bounding_condition.id
trend_capsule_set_json['items'][0]['name'] = metric_output.bounding_condition.name

data_store = '\
{{\
    "version":23,\
    "state":{{\
        "stores":{{\
            "sqWorksheetStore": {0},\
            "sqTrendSeriesStore": {1},\
            "sqTrendCapsuleSetStore": {2}\
        }}\
    }}\
}}'.format(json.dumps(worksheet_store_json), json.dumps(trend_series_json), json.dumps(trend_capsule_set_json))

trend_workstep_input = sdk.WorkstepInputV1()
trend_workstep_input.data = data_store

Now that we have our trend view Workstep defined, let's move on to creating the screenshots.

### Obtaining a Screenshot for an Organizer Topic Document
The Organizer Topics use linked screenshots to display content.  This has the advantage of appearing as immutable content in the Topic document and using the existing Workbench Analysis rendering capabilities to generate the trends and tables.

Screenshots are created using the Jobs API endpoint.  The Jobs API setups the Worksheet containing the scorecard to a specific date range, followed by a "headless capture".  That is, it renders the web page in memory and saves a screenshot without ever displaying it.

The endpoint determines the date range to use for the screenshot from a capsule-producing formula, which is set in `range_formula`.  For scorecards, the `content_selector` must be set to `.screenshotSizeToContent` to ensure that the returned screenshot only shows the scorecard.  Note that when `content_selector` is set to `.screenshotSizeToContent`, the `height` and `width` parameters are ignored, even though they are required.

In the following screenshot, I will use one of the date ranges created in _Creating Organizer Topics using the API_ in which I stored the date range name, id, and start/end ISO8601 timestamps from an API created date range.  You will need these elements when creating and embedding the scorecard in the Topic document. I saved the date ranges in `date_range_info` previously.

In [ ]:
# dictonary with structure {'id': str, 'start_ms': int, 'end_ms': int, 'start_iso': str, 'end_iso': str}
screenshot_date_range = sq_report_store[1]['API Created Date Range 1']

range_formula = 'capsule("{0}Z", "{1}Z")'.format(screenshot_date_range['start_iso'], screenshot_date_range['end_iso'])

jobs_api = sdk.JobsApi(api_client)
metric_screenshot_job_output = jobs_api.get_screenshot(
    worksheet_id=worksheet_output.id,
    workstep_id=analysis_workstep_output.id,
    content_selector='.screenshotSizeToContent',
    range_formula=range_formula,
    height=100, width=100)
print(metric_screenshot_job_output)

The property `screenshot` of `screenshot_job_output` contains the internal URL of our screenshot.

Now that we have the metric screenshot, let's update the workstep and get the trend screenshot.  Note that I'm omitting `content_selector`, updating the `workstep_id`, and setting the `height` and `width` to values I pre-determined.  You'll need to save the height and width values as the jobs endpoint only returns them when it sets them.

In [ ]:
trend_workstep_output = workbook_api.create_workstep(workbook_id=analysis_output.id,
                                                     worksheet_id=worksheet_output.id,
                                                     body=trend_workstep_input)
trend_width = 700
trend_height = int(700./(16./9.)) # make a "widescreen" screenshot
trend_screenshot_job_output = jobs_api.get_screenshot(
    worksheet_id=worksheet_output.id,
    workstep_id=trend_workstep_output.id,
    range_formula=range_formula,
    height=trend_height, width=trend_width)
print(trend_screenshot_job_output)

### Embedding the screenshots into a Topic Document
Seeq content in Topic documents has a specific format, with custom html attributes.  To fill in the attributes, I'm going to create a Jinja2 template.  Typically these templates are hosted in external html files, but we'll create one from a string for clarity.  In addition to the attributes, we want a link to our image's source data to enable drill-down functionality, so I need to create a link to the source Worksheet.  Note that I create a new UUID for the image when I embed it.  This is for frontend content tracking.

In [ ]:
seeq_content_html_tempate = '\
<p>\
<a href="{{worksheet_link}}" class="">\
    <img\
        id="{{image_id}}"\
        data-seeq-content=""\
        class="report-image-border fr-dii specReportSeeqContent fr-draggable"\
        data-seeq-workbookid="{{workbook_id}}"\
        data-seeq-worksheetid="{{worksheet_id}}"\
        data-seeq-workstepid="{{workstep_id}}"\
        data-seeq-datevariableid="{{date_variable_id}}"\
        data-seeq-contentheight="{{content_height}}"\
        data-seeq-contentwidth="{{content_width}}"\
        src="{{screenshot_url}}">\
</a>\
</p>'

#  link creation
protocol = 'http'
host_url = 'localhost'
host_port = 34216

# scorecard link
scorecard_worksheet_link = \
    r'{0}://{1}:{2}/view/worksheet/{3}/{4}?workstepId={5}&;displayRangeStart={6}&;displayRangeEnd={7}&;trendItems='.format(
        protocol, host_url, host_port,
        analysis_output.id,
        worksheet_output.id,
        analysis_workstep_output.id,
        screenshot_date_range['start_iso'],
        screenshot_date_range['end_iso'])

# trend link
trend_worksheet_link = \
    r'{0}://{1}:{2}/view/worksheet/{3}/{4}?workstepId={5}&;displayRangeStart={6}&;displayRangeEnd={7}&;trendItems='.format(
        protocol, host_url, host_port, 
        analysis_output.id,
        worksheet_output.id,
        trend_workstep_output.id,
        screenshot_date_range['start_iso'],
        screenshot_date_range['end_iso'])

jinja_seeq_environment = ji.Environment(loader=ji.BaseLoader).from_string(seeq_content_html_tempate)

scorecard_html = jinja_seeq_environment.render(worksheet_link=scorecard_worksheet_link,
                                               image_id=uuid4(),
                                               workbook_id=analysis_output.id,
                                               worksheet_id=worksheet_output.id,
                                               workstep_id=analysis_workstep_output.id,
                                               date_variable_id=screenshot_date_range['id'],
                                               screenshot_url=metric_screenshot_job_output.screenshot,
                                               content_height=metric_screenshot_job_output.height,
                                               content_width=metric_screenshot_job_output.width)

trend_html = jinja_seeq_environment.render(worksheet_link=trend_worksheet_link,
                                           image_id=uuid4(), workbook_id=analysis_output.id,
                                           worksheet_id=worksheet_output.id,
                                           workstep_id=trend_workstep_output.id,
                                           date_variable_id=screenshot_date_range['id'],
                                           screenshot_url=trend_screenshot_job_output.screenshot,
                                           content_height=trend_height,
                                           content_width=trend_width)

# now to combine them
seeq_html = scorecard_html + trend_html

The final step is to update our Annotation with the html we just created.  Details on these calls can be found in the companion how to _Creating Organizer Topics using the API_.  I will be reusing the `document_output` as well as the `annotation_output` items from that how-to.  These contain the IDs of the Topic document and Annotation that we created using the API.

In [ ]:
seeq_data_annotation_input = sdk.AnnotationInputV1()
seeq_data_annotation_input.name = "Unnamed"
seeq_data_annotation_input.type = "Report"
seeq_data_annotation_input.createdById = "Seth Gilchrist"

seeq_data_annotation_interested_input = sdk.AnnotationInterestInputV1()
seeq_data_annotation_interested_input.interest_id = document_output.id
seeq_data_annotation_input.interests = [annotations_interested_input]

seeq_data_annotation_input.document = seeq_html

seeq_data_annotation_update_output = annotations_api.update_annotation(
    id=annotation_output.id, body=seeq_data_annotation_input)
print(seeq_data_annotation_update_output)

You can now go to the Topic, open the document and find the scorecard, with all the Seeq frontend tools at your disposal.

## Taking the Next Step 
Using the same techniques, we can embed all the different kinds of Seeq content in to Topic documents.

To explore different kinds of Seeq content, I encourage you to use the "Inspect" capability of Chrome to examine the html that the frontend uses and creates, and also to take a look at the different data stores in worksteps that you are interested in.  One useful method of understanding the effects of different operations is to use a diff tool (e.g., Meld) to examine changes made to the data store between operations.

Once you have a feel for how the different operations and views manifest in the data store and html, it becomes possible to highly customize your views using the API and create a wide variety of content.

As a final note, I'd like to point out that, as one of our developers, says, "The frontend eats it's own dog food - everything the frontend does is done through the API." So the sky's the limit once you understand how data is stored and objects are represented.

## Contact Us!
If you have questions, comments, or need more help or information, please reach out to your Seeq representative to get a team or 1:1 training on these tools.  We're here to help!